# Minimal notebook for viewing transcript locations in Napari

This is a minimal example notebook using the [Napari](https://napari.org) Viewer. 

## Requirements:
1. Install Napari, following their instructions [here](https://napari.org/stable/#installation)
2. make sure your environment has the packages used below
3. access to mRNA location data in a `pandas`-readable format
4. optional image for background.  if using an image, mRNA location data must currently be in pixel coordinates

## Notes:
1. Future versions will integrate with the `sawg` `SpotTable` infrastructure
2. Small differences in file formats can be handled here, but major differences (e.g. image transformations, more complex file formats for coordinates, etc) will require some more work.
3. Napari can have performance slow-downs with many layers (100 genes) and/or many (1M+) transcripts

In [1]:
from napari import Viewer
import pandas as pd
import numpy as np
from pathlib import Path
import imageio



from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Data file paths:

In [2]:
# transcript location file:

transcript_filepath = Path("/home/imaging_mfish/Desktop/32684-Slide1_A1-1_results.csv")
image_filepath = Path("/home/imaging_mfish/Desktop/32684-Slide1_A1-1_DAPI.tiff")

# Read mRNA location information into a `pandas.DataFrame`

In [3]:


transcript_df  = pd.read_csv(transcript_filepath)
image = imageio.imread(image_filepath)



# take a look at the transcript location `DataFrame`



In [7]:
transcript_df.head(10)

,x,y,z,gene
0,2339,529,23,Aif1l
1,3154,361,23,Aif1l
2,3156,376,23,Aif1l
3,3468,301,23,Aif1l
4,2772,771,24,Aif1l
5,2806,1428,24,Aif1l
6,2858,1419,24,Aif1l
7,3289,856,24,Aif1l
8,3361,483,24,Aif1l
9,2374,1284,25,Aif1l


## identify which columns in the dataframe should be used in napari
For this basic example, these are the pixel coordinates (note napari used row-column ordering for image coordinates)

In [9]:
# relevant columns are format-specific...  here I'm reading in Resolve data:

napari_x_column = "y"
napari_y_column = "x"
gene_column = "gene"



# Create a napari `Viewer`
This should make a new napari viewer window.  If this doesn't work, make sure you have correctly installed `napari` and can make a viewer from the command line using the command `napari`

In [4]:

v = Viewer()

/home/imaging_mfish/anaconda3/envs/merscope_notebooks/lib/python3.9/site-packages/napari/_qt/qt_viewer.py:344: UserWarning: Attempting to unbind an action which does not exists (napari:also_select_layer_above), this may have no effects. This can happen if your settings are out of date, if you upgraded napari, upgraded or deactivated a plugin, or made a typo in in your custom keybinding.
  action_manager.unbind_shortcut(action)
/home/imaging_mfish/anaconda3/envs/merscope_notebooks/lib/python3.9/site-packages/napari/_qt/qt_viewer.py:344: UserWarning: Attempting to unbind an action which does not exists (napari:also_select_layer_below), this may have no effects. This can happen if your settings are out of date, if you upgraded napari, upgraded or deactivated a plugin, or made a typo in in your custom keybinding.
  action_manager.unbind_shortcut(action)
/home/imaging_mfish/anaconda3/envs/merscope_notebooks/lib/python3.9/site-packages/napari/_qt/qt_viewer.py:344: UserWarning: Attempting to 

In [10]:
genes = list(transcript_df[gene_column].unique())
genes

['Aif1l',
 'Maf',
 'Adcy8',
 'Slc17a6',
 'Dpyd',
 'Robo1',
 'St6galnac5',
 'Pecam1',
 'Grik1',
 'Il23a',
 'Adam11',
 'Abca8a',
 'Opalin',
 'Wnk2',
 'Slc17a7',
 'Unc13c',
 'Gad2',
 'Grin2a',
 'Hdac9',
 'Ccnd2',
 'Mdga2',
 'Crym',
 'Cd38',
 'Sulf2',
 'Slc6a9',
 'Kcnab1',
 'Efna5',
 'Cdkn1a',
 'Grin3a',
 'Adgrl3',
 'Neu4',
 'Mlc1',
 'Vit',
 'Hmgcs2',
 'Acss1',
 'Ust',
 'Klhl29',
 'Nrg3',
 'Nav3',
 'Sox10',
 'Pdgfra',
 'Des',
 'Hopx',
 'Pdlim2',
 'Shisa9',
 'Gfap',
 'Itgb5',
 'Slc6a6',
 'Brinp3',
 'Pcolce2',
 'Rasgrf2',
 'Dcn',
 'Kcnj8',
 'Art3',
 'Dgat2',
 'Gad1',
 'Fmo2',
 'Nrgn',
 'Gstm1',
 'Apod',
 'Rgs9',
 'Jph4',
 'Ildr2',
 'Cdh8',
 'Spock3',
 'Dpysl3',
 'Igfbp2',
 'Mrc1',
 'Ctss',
 'Cnih2',
 'Sema3g',
 'Aqp4',
 'Nkx2-9',
 'Inpp4b',
 'Susd5',
 'Snhg11',
 'Svop',
 'Prom1',
 'Gria1',
 'Auts2',
 'Il33',
 'Ttll3',
 'Cyp26b1',
 'Dock5',
 'Slc25a18',
 'Nr6a1',
 'Nkx6-2',
 'Pdgfrb',
 'Arhgap29',
 'Enpp6',
 'Adamts18',
 'Gadd45b',
 'Itga6',
 'Cd24a',
 'Sgcd',
 'Ccl3',
 'Ephb1',
 'Snca',
 'Cc

# Select a subset of genes to show. 
Transcripts from all genes will be colored gray and can be subsampled to keep visualization responsive

In [11]:
genes_to_show =["Gad1", "Gad2", "Slc17a7","Apod","Abca8a", "Gfap"]

genes_to_show = [g for g in genes_to_show if g in genes]


# I'm using random colors here for transcripts:

color_value_bias = 0  #use color_value_bias = -.3  for white background, .3 for dark background 
random_colors = np.clip(np.random.rand(1000,4)+color_value_bias, a_max=1.0, a_min=0)
random_colors[:,3]=1.0


# all genes, including those not in `genes_to_show` will be subsampled by this factor
subsample_background_points = 5


# genes in `genes_to_show` will be subsampled by this factor:
subsample_target_points = 1


In [12]:
# add image:
# this usage assumes the transcript locations are reported in image pixel coordinates
img_layer = v.add_image(image)

<Image layer 'image' at 0x7fa303fb2d90>

In [13]:

# make a 2D numpy array.  This isn't strictly necessary to make a copy here, but it may help for large and complicated DataFrames that have 
napari_coordinates = transcript_df.loc[:,[napari_x_column, napari_y_column]].to_numpy()




color_iter=0
v.add_points(napari_coordinates[::subsample_background_points,:],
                size=2,
                edge_width=0,
                 name = "all transcripts, subsampled x"+str(subsample_background_points), face_color=[.5, .5, .5, 1.])



for gene in genes_to_show:
    genemask = transcript_df[gene_column]==gene
    if np.sum(genemask)>100:
        
        v.add_points(napari_coordinates[genemask,:][::subsample_target_points,:],
                size=3,
                edge_width=0,
                name = gene, face_color=random_colors[color_iter,:])
    color_iter +=1

/home/imaging_mfish/anaconda3/envs/merscope_notebooks/lib/python3.9/site-packages/numpy/core/numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
/home/imaging_mfish/anaconda3/envs/merscope_notebooks/lib/python3.9/site-packages/numpy/core/numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
/home/imaging_mfish/anaconda3/envs/merscope_notebooks/lib/python3.9/site-packages/numpy/core/numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
/home/imaging_mfish/anaconda3/envs/merscope_notebooks/lib/python3.9/site-packages/numpy/core/numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar in